# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [508]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Generate the mathematical model

In [555]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [556]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Sets and Indices

*  $I$: Set of thermal units, $I = \{1, 2, 3, 4, 5, 6\}$
* $T$: Set of time periods, $T = \{1, 2, ..., 15\}$

### Parameters

* $SU_i$: Startup cost for thermal unit $i \in I$ (€).
   * $SU_1 = 10324$, $SU_2 = 5678$, $SU_3 = 7802$, $SU_4 = 12899$, $SU_5 = 4596$, $SU_6 = 9076$.
* $SD_i$: Shutdown cost for thermal unit $i \in I$ (€).
   * $SD_1 = 2673$, $SD_2 = 5893$, $SD_3 = 982$, $SD_4 = 6783$, $SD_5 = 2596$, $SD_6 = 3561$. 
* $FC_i$: Fixed cost for running thermal unit $i \in I$ (€).
* $VC_{i,t}$: Variable cost for thermal unit $i \in I$ at time $t \in T$ (€/MWh).
* $Pmin_i$: Minimum power output of thermal unit $i \in I$ (MW).
* $Pmax_i$: Maximum power output of thermal unit $i \in I$ (MW).
* $RU_i$: Maximum power increment for thermal unit $i \in I$ (MW).
* $RD_i$: Maximum power decrement for thermal unit $i \in I$ (MW).
* $D_t$: Total power demand at time $t \in T$ (MW). 

### Decision Variables

* $p_{i,t}$: Power output of thermal unit $i \in I$ at time $t \in T$ (MW).
* $u_{i,t}$: Binary variable indicating whether thermal unit $i \in I$ is running at time $t \in T$.  ($u_{i,t} = 1$ if running, $0$ otherwise).
* $su_{i,t}$: Binary variable indicating whether thermal unit $i \in I$ is started up at time $t \in T$. ($su_{i,t} = 1$ if started up, $0$ otherwise).
* $sd_{i,t}$: Binary variable indicating whether thermal unit $i \in I$ is shut down at time $t \in T$. ($sd_{i,t} = 1$ if shut down, $0$ otherwise). 

### Objective Function

Minimize the total cost, which includes the fixed cost, the variable cost, the startup cost and the shutdown cost.
 
 $$
 \text{Minimize} \sum_{i \in I} \sum_{t \in T} (FC_i \cdot u_{i,t} + VC_{i,t} \cdot p_{i,t} + SU_i \cdot su_{i,t} + SD_i \cdot sd_{i,t})
 $$

### Constraints

1. **Power balance:** The total power generated must meet the demand at each time period, with a 10% safety margin.
   $$ \sum_{i \in I} p_{i,t} \ge 1.1 \cdot D_t,  \forall t \in T $$
2. **Operational limits:** The power output of each unit must be within its minimum and maximum limits.
   $$ Pmin_i \cdot u_{i,t} \le p_{i,t} \le Pmax_i \cdot u_{i,t}, \forall i \in I, \forall t \in T $$
3. **Ramp-up and ramp-down limits:**  
    $$ p_{i,t} - p_{i, t-1} \le RU_i, \forall i \in I, \forall t \in T, t>1 $$
    $$ p_{i, t-1} - p_{i,t} \le RD_i, \forall i \in I, \forall t \in T, t>1 $$
4. **Startup and shutdown logic:**
    $$ su_{i,t} \ge u_{i,t} - u_{i,t-1}, \forall i \in I, \forall t \in T, t>1 $$
    $$ sd_{i,t} \ge u_{i,t-1} - u_{i,t}, \forall i \in I, \forall t \in T, t>1 $$

This model defines the decision variables, the objective function, and the constraints necessary to determine the optimal power output schedule for each thermal unit while minimizing cost and fulfilling the demand and security requirements.


## 3. Generate the pyomo code

In [557]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [558]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data
# Set of thermal units
I = [1, 2, 3, 4, 5, 6]
# Set of time periods
T = list(range(1, 16))

# Parameters
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 120, 5: 70, 6: 110} # Sample data
VC = {(i, t): 0.1 * i * t for i in I for t in T} # Sample data
Pmin = {1: 10, 2: 8, 3: 12, 4: 15, 5: 9, 6: 11} # Sample data
Pmax = {1: 50, 2: 40, 3: 60, 4: 75, 5: 45, 6: 55} # Sample data
RU = {1: 20, 2: 15, 3: 25, 4: 30, 5: 18, 6: 22} # Sample data
RD = {1: 25, 2: 20, 3: 30, 4: 35, 5: 22, 6: 27} # Sample data
D = {t: 100 + 5 * t for t in T} # Sample data

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.p = pyo.Var(I, T, within=pyo.NonNegativeReals)
model.u = pyo.Var(I, T, within=pyo.Binary)
model.su = pyo.Var(I, T, within=pyo.Binary)
model.sd = pyo.Var(I, T, within=pyo.Binary)

# Objective function
def objective_function(model):
    return sum(
        FC[i] * model.u[i, t] + VC[i, t] * model.p[i, t] + SU[i] * model.su[i, t] + SD[i] * model.sd[i, t]
        for i in I
        for t in T
    )

model.obj = pyo.Objective(rule=objective_function, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance(model, t):
    return sum(model.p[i, t] for i in I) >= 1.1 * D[t]

model.power_balance = pyo.Constraint(T, rule=power_balance)

# Operational limits
def operational_limits(model, i, t):
    return Pmin[i] * model.u[i, t] <= model.p[i, t] <= Pmax[i] * model.u[i, t]

model.operational_limits = pyo.Constraint(I, T, rule=operational_limits)

# Ramp-up and ramp-down limits
def ramp_up_limits(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t - 1] <= RU[i]
    else:
        return pyo.Constraint.Skip

model.ramp_up_limits = pyo.Constraint(I, T, rule=ramp_up_limits)

def ramp_down_limits(model, i, t):
    if t > 1:
        return model.p[i, t - 1] - model.p[i, t] <= RD[i]
    else:
        return pyo.Constraint.Skip

model.ramp_down_limits = pyo.Constraint(I, T, rule=ramp_down_limits)

# Startup and shutdown logic
def startup_logic(model, i, t):
    if t > 1:
        return model.su[i, t] >= model.u[i, t] - model.u[i, t - 1]
    else:
        return pyo.Constraint.Skip

model.startup_logic = pyo.Constraint(I, T, rule=startup_logic)

def shutdown_logic(model, i, t):
    if t > 1:
        return model.sd[i, t] >= model.u[i, t - 1] - model.u[i, t]
    else:
        return pyo.Constraint.Skip

model.shutdown_logic = pyo.Constraint(I, T, rule=shutdown_logic)

# Solve the model
solver = pyo.SolverFactory('cbc')  # You can choose a different solver if needed
solver.solve(model)

# Print the results
print("Optimal Solution:")
for t in T:
    print(f"Time period {t}:")
    for i in I:
        print(f"  Unit {i}: Power output = {pyo.value(model.p[i, t])}, Status = {'Running' if pyo.value(model.u[i, t]) else 'Offline'}")

print(f"Total cost: {pyo.value(model.obj)}")
```

This code defines the sets, parameters, variables, objective function, and constraints of the optimization problem. Please note that the values for `FC`, `VC`, `Pmin`, `Pmax`, `RU`, `RD`, and `D` are sample data and need to be replaced with your actual data. 


## 4. Input problem data and try running the generated code

In [559]:
import pyomo.environ as pyo

# Sample data
# Set of thermal units
I = [1, 2, 3, 4, 5, 6]
# Set of time periods
T = list(range(1, 16))

# Parameters
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 120, 5: 70, 6: 110} # Sample data
VC = {(i, t): 0.1 * i * t for i in I for t in T} # Sample data
Pmin = {1: 10, 2: 8, 3: 12, 4: 15, 5: 9, 6: 11} # Sample data
Pmax = {1: 50, 2: 40, 3: 60, 4: 75, 5: 45, 6: 55} # Sample data
RU = {1: 20, 2: 15, 3: 25, 4: 30, 5: 18, 6: 22} # Sample data
RD = {1: 25, 2: 20, 3: 30, 4: 35, 5: 22, 6: 27} # Sample data
D = {t: 100 + 5 * t for t in T} # Sample data

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.p = pyo.Var(I, T, within=pyo.NonNegativeReals)
model.u = pyo.Var(I, T, within=pyo.Binary)
model.su = pyo.Var(I, T, within=pyo.Binary)
model.sd = pyo.Var(I, T, within=pyo.Binary)

# Objective function
def objective_function(model):
    return sum(
        FC[i] * model.u[i, t] + VC[i, t] * model.p[i, t] + SU[i] * model.su[i, t] + SD[i] * model.sd[i, t]
        for i in I
        for t in T
    )

model.obj = pyo.Objective(rule=objective_function, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance(model, t):
    return sum(model.p[i, t] for i in I) >= 1.1 * D[t]

model.power_balance = pyo.Constraint(T, rule=power_balance)

# Operational limits
def operational_limits(model, i, t):
    return Pmin[i] * model.u[i, t] <= model.p[i, t] <= Pmax[i] * model.u[i, t]

model.operational_limits = pyo.Constraint(I, T, rule=operational_limits)

# Ramp-up and ramp-down limits
def ramp_up_limits(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t - 1] <= RU[i]
    else:
        return pyo.Constraint.Skip

model.ramp_up_limits = pyo.Constraint(I, T, rule=ramp_up_limits)

def ramp_down_limits(model, i, t):
    if t > 1:
        return model.p[i, t - 1] - model.p[i, t] <= RD[i]
    else:
        return pyo.Constraint.Skip

model.ramp_down_limits = pyo.Constraint(I, T, rule=ramp_down_limits)

# Startup and shutdown logic
def startup_logic(model, i, t):
    if t > 1:
        return model.su[i, t] >= model.u[i, t] - model.u[i, t - 1]
    else:
        return pyo.Constraint.Skip

model.startup_logic = pyo.Constraint(I, T, rule=startup_logic)

def shutdown_logic(model, i, t):
    if t > 1:
        return model.sd[i, t] >= model.u[i, t - 1] - model.u[i, t]
    else:
        return pyo.Constraint.Skip

model.shutdown_logic = pyo.Constraint(I, T, rule=shutdown_logic)

# Solve the model
solver = pyo.SolverFactory('cbc')  # You can choose a different solver if needed
solver.solve(model)

# Print the results
print("Optimal Solution:")
for t in T:
    print(f"Time period {t}:")
    for i in I:
        print(f"  Unit {i}: Power output = {pyo.value(model.p[i, t])}, Status = {'Running' if pyo.value(model.u[i, t]) else 'Offline'}")

print(f"Total cost: {pyo.value(model.obj)}")

ERROR: Rule failed when generating expression for Constraint
operational_limits with index (1, 1): PyomoException: Cannot convert non-
constant Pyomo expression (10*u[1,1]  <=  p[1,1]) to bool. This error is
usually caused by using a Var, unit, or mutable Param in a Boolean context
such as an "if" statement, or when checking container membership or equality.
For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'operational_limits' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression
        (10*u[1,1]  <=  p[1,1]) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
       

PyomoException: Cannot convert non-constant Pyomo expression (10*u[1,1]  <=  p[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [560]:
print(response.text)

### Sets and Indices

*  $I$: Set of thermal units, $I = \{1, 2, 3, 4, 5, 6\}$
* $T$: Set of time periods, $T = \{1, 2, ..., 15\}$

### Parameters

* $SU_i$: Startup cost for thermal unit $i \in I$ (€).
   * $SU_1 = 10324$, $SU_2 = 5678$, $SU_3 = 7802$, $SU_4 = 12899$, $SU_5 = 4596$, $SU_6 = 9076$.
* $SD_i$: Shutdown cost for thermal unit $i \in I$ (€).
   * $SD_1 = 2673$, $SD_2 = 5893$, $SD_3 = 982$, $SD_4 = 6783$, $SD_5 = 2596$, $SD_6 = 3561$. 
* $FC_i$: Fixed cost for running thermal unit $i \in I$ (€).
* $VC_{i,t}$: Variable cost for thermal unit $i \in I$ at time $t \in T$ (€/MWh).
* $Pmin_i$: Minimum power output of thermal unit $i \in I$ (MW).
* $Pmax_i$: Maximum power output of thermal unit $i \in I$ (MW).
* $RU_i$: Maximum power increment for thermal unit $i \in I$ (MW).
* $RD_i$: Maximum power decrement for thermal unit $i \in I$ (MW).
* $D_t$: Total power demand at time $t \in T$ (MW). 

### Decision Variables

* $p_{i,t}$: Power output of thermal unit $i \in I$ at time $t \

In [561]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data
# Set of thermal units
I = [1, 2, 3, 4, 5, 6]
# Set of time periods
T = list(range(1, 16))

# Parameters
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 120, 5: 70, 6: 110} # Sample data
VC = {(i, t): 0.1 * i * t for i in I for t in T} # Sample data
Pmin = {1: 10, 2: 8, 3: 12, 4: 15, 5: 9, 6: 11} # Sample data
Pmax = {1: 50, 2: 40, 3: 60, 4: 75, 5: 45, 6: 55} # Sample data
RU = {1: 20, 2: 15, 3: 25, 4: 30, 5: 18, 6: 22} # Sample data
RD = {1: 25, 2: 20, 3: 30, 4: 35, 5: 22, 6: 27} # Sample data
D = {t: 100 + 5 * t for t in T} # Sample data

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.p = pyo.Var(I, T, within=pyo.NonNegativeReals)
model.u = pyo.Var(I, T, within=pyo.Binary)
model.su = pyo.Var(I, T, within=pyo.Binary)
model.sd = pyo.Var(I, T, within=pyo.Binary)

# Objective function
def objective_function(